In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as py

Read Data

In [2]:
car_price_data=pd.read_csv("audi.csv")
car_price_data

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0
...,...,...,...,...,...,...,...,...,...
10663,A3,2020,16999,Manual,4018,Petrol,145,49.6,1.0
10664,A3,2020,16999,Manual,1978,Petrol,150,49.6,1.0
10665,A3,2020,17199,Manual,609,Petrol,150,49.6,1.0
10666,Q3,2017,19499,Automatic,8646,Petrol,150,47.9,1.4


Check for nulls

In [3]:
car_price_data.isnull().sum()

model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

No null values present

Label Encode model and fuel type and onehotencode trnasmission

Create X and Y variables

In [4]:
X = car_price_data.iloc[:,[0,1,3,4,5,6,7,8]].values
Y = car_price_data.iloc[:,[2]].values

In [5]:
from sklearn.preprocessing import LabelEncoder
l1=LabelEncoder()
X[:,0]=l1.fit_transform(X[:,0])
l2=LabelEncoder()
X[:,4]=l2.fit_transform(X[:,4])



In [6]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
col_trans=ColumnTransformer(transformers=
                            [('encoder',OneHotEncoder(),[2])],remainder='passthrough')
X=col_trans.fit_transform(X)
X

array([[0.0, 1.0, 0.0, ..., 150, 55.4, 1.4],
       [1.0, 0.0, 0.0, ..., 20, 64.2, 2.0],
       [0.0, 1.0, 0.0, ..., 30, 55.4, 1.4],
       ...,
       [0.0, 1.0, 0.0, ..., 150, 49.6, 1.0],
       [1.0, 0.0, 0.0, ..., 150, 47.9, 1.4],
       [0.0, 1.0, 0.0, ..., 150, 47.9, 1.4]], dtype=object)

Standardization

In [7]:
from sklearn.preprocessing import StandardScaler
std_sc=StandardScaler()
X=std_sc.fit_transform(X)
X

array([[-0.58326752,  1.2007284 , -0.71233307, ...,  0.35714729,
         0.35755001, -0.88021837],
       [ 1.71447913, -0.83282781, -0.71233307, ..., -1.57832278,
         1.03713001,  0.11492465],
       [-0.58326752,  1.2007284 , -0.71233307, ..., -1.42944047,
         0.35755001, -0.88021837],
       ...,
       [-0.58326752,  1.2007284 , -0.71233307, ...,  0.35714729,
        -0.09035499, -1.54364705],
       [ 1.71447913, -0.83282781, -0.71233307, ...,  0.35714729,
        -0.22163749, -0.88021837],
       [-0.58326752,  1.2007284 , -0.71233307, ...,  0.35714729,
        -0.22163749, -0.88021837]])

Train Test Split

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

Import model libraies 

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
%pip install catboost
from catboost import CatBoostRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import  ExtraTreesRegressor


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


import metrics

In [10]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

Create models

In [11]:
model1=LinearRegression()
model2=RandomForestRegressor()
n_estimators = [int(x) for x in np.linspace(start = 80, stop = 1500, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(6, 45, num = 5)]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10]
rand_grid={'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}


model3=RandomizedSearchCV(estimator=model2,param_distributions=rand_grid,scoring='neg_mean_squared_error',n_iter=3,cv=3,random_state=42, n_jobs = -1)
model4=CatBoostRegressor()
model5=GaussianNB()

Fit the models

In [12]:
model1.fit(x_train,y_train)
model2.fit(x_train,y_train)
model3.fit(x_train,y_train)
model4.fit(x_train,y_train)
model5.fit(x_train,y_train)

c:\Users\Kaustubha Ram\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Kaustubha Ram\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
3 fits failed out of a total of 9.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Kaustubha Ram\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888,

Learning rate set to 0.057452
0:	learn: 11184.0694779	total: 152ms	remaining: 2m 31s
1:	learn: 10690.2138498	total: 157ms	remaining: 1m 18s
2:	learn: 10221.1011877	total: 160ms	remaining: 53.1s
3:	learn: 9778.8067263	total: 164ms	remaining: 40.8s
4:	learn: 9382.6454054	total: 167ms	remaining: 33.2s
5:	learn: 8989.9290587	total: 170ms	remaining: 28.1s
6:	learn: 8628.4892653	total: 173ms	remaining: 24.5s
7:	learn: 8280.9179062	total: 176ms	remaining: 21.8s
8:	learn: 7950.5476053	total: 179ms	remaining: 19.7s
9:	learn: 7636.7735365	total: 182ms	remaining: 18s
10:	learn: 7349.6084699	total: 186ms	remaining: 16.7s
11:	learn: 7084.0457263	total: 189ms	remaining: 15.6s
12:	learn: 6842.7060927	total: 192ms	remaining: 14.6s
13:	learn: 6608.0244743	total: 195ms	remaining: 13.7s
14:	learn: 6387.7285726	total: 198ms	remaining: 13s
15:	learn: 6170.1354894	total: 204ms	remaining: 12.5s
16:	learn: 5973.9133066	total: 207ms	remaining: 12s
17:	learn: 5780.4713044	total: 211ms	remaining: 11.5s
18:	learn

c:\Users\Kaustubha Ram\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

Check metrics for each model

In [13]:
print("Linear Regresion: ")
y_train_pred=model1.predict(x_train)
y_test_pred=model1.predict(x_test)
print("Train Data r2_score: ",r2_score(y_train,y_train_pred))
print("Test Data r2_score: ",r2_score(y_test,y_test_pred))
print("mean_absolute_error: ",mean_absolute_error(y_test,y_test_pred))
print("mean_squared_error: ",mean_squared_error(y_test,y_test_pred))



Linear Regresion: 
Train Data r2_score:  0.7904877913746597
Test Data r2_score:  0.7916097559453965
mean_absolute_error:  3381.4951071085525
mean_squared_error:  28636156.321987584


In [14]:
print("Random Forest Regresion: ")
y_train_pred=model2.predict(x_train)
y_test_pred=model2.predict(x_test)
print("Train Data r2_score: ",r2_score(y_train,y_train_pred))
print("Test Data r2_score: ",r2_score(y_test,y_test_pred))
print("mean_absolute_error: ",mean_absolute_error(y_test,y_test_pred))
print("mean_squared_error: ",mean_squared_error(y_test,y_test_pred))

Random Forest Regresion: 
Train Data r2_score:  0.9933580381463298
Test Data r2_score:  0.9564924309272625
mean_absolute_error:  1529.3415569483413
mean_squared_error:  5978636.643038475


In [15]:
print("RandomizedSearchCV: ")
y_train_pred=model3.predict(x_train)
y_test_pred=model3.predict(x_test)
print("Train Data r2_score: ",r2_score(y_train,y_train_pred))
print("Test Data r2_score: ",r2_score(y_test,y_test_pred))
print("mean_absolute_error: ",mean_absolute_error(y_test,y_test_pred))
print("mean_squared_error: ",mean_squared_error(y_test,y_test_pred))

RandomizedSearchCV: 
Train Data r2_score:  0.9814845984965838
Test Data r2_score:  0.9583954079122329
mean_absolute_error:  1503.7913495161433
mean_squared_error:  5717137.134431544


In [16]:
print("CatBoost Regression: ")
y_train_pred=model4.predict(x_train)
y_test_pred=model4.predict(x_test)
print("Train Data r2_score: ",r2_score(y_train,y_train_pred))
print("Test Data r2_score: ",r2_score(y_test,y_test_pred))
print("mean_absolute_error: ",mean_absolute_error(y_test,y_test_pred))
print("mean_squared_error: ",mean_squared_error(y_test,y_test_pred))

CatBoost Regression: 
Train Data r2_score:  0.9771653776071909
Test Data r2_score:  0.9641612028134969
mean_absolute_error:  1453.5230700749125
mean_squared_error:  4924824.591864283


In [17]:
print("Naive Gaussian Bayes: ")
y_train_pred=model5.predict(x_train)
y_test_pred=model5.predict(x_test)
print("Train Data r2_score: ",r2_score(y_train,y_train_pred))
print("Test Data r2_score: ",r2_score(y_test,y_test_pred))
print("mean_absolute_error: ",mean_absolute_error(y_test,y_test_pred))
print("mean_squared_error: ",mean_squared_error(y_test,y_test_pred))
print("\n\n")

Naive Gaussian Bayes: 
Train Data r2_score:  0.8209180329257256
Test Data r2_score:  0.6567724626859931
mean_absolute_error:  4117.880037488285
mean_squared_error:  47164959.45923149





CatBoost Regression has highest accuracy.Saving model to disk

In [21]:
import pickle 
pickle.dump(model4, open('car_price_data_model.pkl','wb'))
model=pickle.load(open('car_price_data_model.pkl','rb'))
print (model.predict (x_train))


[14256.69390956 29475.32741129 11911.12888914 ... 21406.16876331
 17189.65426694 44839.56117235]
